# Preprocessing of individual datasets

Import packages... 
- [scanpy](https://scanpy.readthedocs.io/en/stable/index.html)


In [1]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd

import gc
import os

In [2]:
WORKDIR="/workdir/dwm269/scMuscle2"
os.chdir(WORKDIR)

In [3]:
from scripts.py.scantils import *
# from scripts.py.scanplots import *

In [4]:
gc.enable()

In [5]:
DATADIR="/workdir/dwm269/scMuscle2/data/align_out"

Scanpy settings info [[link](https://scanpy.readthedocs.io/en/stable/generated/scanpy._settings.ScanpyConfig.html)]

In [10]:
sc.settings.cachedir = './data/cache/'
sc.settings.n_jobs=16

Load metadata

In [8]:
meta = pd.read_csv("scMuscle2_metadata_v1-0.csv")
meta = meta.loc[meta["include"],] # remove unwanted metadata
meta = meta[[x in ["fastq", "bam"] for x in meta["file.format"]]] # remove samples w/ download issues

meta = meta[[x in ["muscle", "tendon"] for x in meta["tissue"]]] # subset by tissue 
meta = meta[[x in ["Homo sapiens"] for x in meta["species"]]] # subset by species , "Mus musculus"
meta = meta.iloc[[8,12,23,34,45],] # subset by row index

meta = meta[[x != "" for x in meta["GSM.accession"]]] # 
meta = meta[[x != "" for x in meta["sample"]]] # 

meta.index = list(range(0, len(meta))) # reset row indices
# meta

In [9]:
meta.tissue.value_counts()

muscle    4
tendon    1
Name: tissue, dtype: int64

Read in count data & initialize anndata objects. Also add metadata to each object

In [10]:
# try:
#     scm_list.index
# except:
scm_list = pd.Series(
    index = meta["GSM.accession"],
    data = [""]*meta.shape[0]
)

In [11]:
for i in range(0,meta.shape[0]):
    if os.path.exists(DATADIR+"/"+meta["GSM.accession"][i]+"/STARsolo/Solo.out/GeneFull/filtered/matrix.mtx.gz"):
        print("Sample: " + meta["GSM.accession"][i]) 
#         try:
        scm_list[i] = sc.read_10x_mtx(
                path=DATADIR+"/"+meta["GSM.accession"][i]+"/STARsolo/Solo.out/GeneFull/filtered", 
                var_names='gene_symbols',
                make_unique=True,
                cache=True
            )
          
        scm_list[i].var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

        for j in range(0,meta.shape[1]): #metadata features
            scm_list[i].obs[meta.columns[j]] = meta.iloc[i,j]

        scm_list[i].layers['counts'] = scm_list[i].X # save counts as a layer for future plotting

        print("     Loaded " + str(scm_list[i].shape[0]) + " cells and " + str(scm_list[i].shape[1]) + " genes...")
    else:
        print("Can't find counts for " + str(meta["GSM.accession"][i]) + "...")
#         except:
#             print("Exception with " + meta["sample"][i])

Sample: GSM5848681


/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


     Loaded 5499 cells and 62703 genes...
Sample: GSM4272893


/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


     Loaded 2393 cells and 62703 genes...
Sample: GSM5098738


/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


     Loaded 7737 cells and 62703 genes...
Sample: GSM5098749


/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


     Loaded 3802 cells and 62703 genes...
Sample: GSM4743496
     Loaded 3963 cells and 62703 genes...


/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


Add ambient-RNA-scrubbed counts

In [ ]:
#TODO

In [21]:
gc.collect()

1422

QC filter and preprocess individual datasets

In [28]:
# gene/transcript, mito filters count filters
for i in range(0,meta.shape[0]):
    try:
        print(meta["GSM.accession"][i] + ': ' + str(scm_list[i].shape[0]) + " cells and " + str(scm_list[i].shape[1]) +' features...')  

        # Hard filters for feature and UMI counts
        sc.pp.filter_cells(
            scm_list[i],
            min_genes=500
        )
        sc.pp.filter_cells(
            scm_list[i], 
            min_counts=1000
        )

        scm_list[i].var['mito'] = scm_list[i].var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
        sc.pp.calculate_qc_metrics(
            scm_list[i], 
            qc_vars=['mito'], 
            percent_top=None, 
            log1p=False, 
            inplace=True
        )  

        # QC filter(s)
        scm_list[i] = scm_list[i][scm_list[i].obs.pct_counts_mito < 40, :] 

        print('After filtering: ' + str(scm_list[i].shape[0]) + " cells and " + str(scm_list[i].shape[1]) +' features...')  
    
    except:
            print("Exception with " + str(meta["GSM.accession"][i]))
        
    print(" ")

GSM4743488: 4788 cells and 61860 features...
After filtering: 3004 cells and 61860 features...
 
GSM4743489: 3153 cells and 61860 features...
After filtering: 1814 cells and 61860 features...
 
GSM4743490: 3050 cells and 61860 features...
After filtering: 2232 cells and 61860 features...
 
GSM4743491: 2505 cells and 61860 features...
After filtering: 1870 cells and 61860 features...
 
GSM4743492: 2174 cells and 61860 features...
After filtering: 1936 cells and 61860 features...
 
GSM4743493: 2643 cells and 61860 features...
After filtering: 2485 cells and 61860 features...
 
GSM4743494: 2706 cells and 61860 features...
After filtering: 2578 cells and 61860 features...
 
GSM4743495: 3207 cells and 61860 features...
After filtering: 2532 cells and 61860 features...
 
GSM5848681: 5500 cells and 61860 features...
After filtering: 5274 cells and 61860 features...
 
GSM5848680: 5875 cells and 61860 features...
After filtering: 5234 cells and 61860 features...
 
GSM5848679: 6432 cells and 618

Add metadata to each object

## Doublet removal via Scrublet

Estimate doublet scores

In [ ]:
# sc.settings.set_figure_params(
#     fontsize=8
# )

# for i in range(0,meta.shape[0]):
#     try:
#         sc.external.pp.scrublet(
#             scm_list[i]
#         )
#         sc.external.pl.scrublet_score_distribution(
#             scm_list[i],
#             figsize =[6,2.25]
#         )
            
#     except:
#             print("Exception with " + meta["sample"][i])
        
#     print(" ")

Estimate doublet score cutoff values for each sample

In [ ]:
# cutoff_threshold = [
#     0.63, 0.63, 
#     0.59, #D4_200um
#     0.52, 
#     0.6, #D4_1000um
#     0.24, 
#     0.2, 0.24,
# #     0.58, # D20_600um
#     0.63, # D21_200um
#     0.18,
#     0.18 #D21_1000um
# ]

In [ ]:
# scm_list

In [ ]:
# print("Final cell & feature counts:\n")
# for i in range(0,meta.shape[0]):    
#     scm_list[i] = scm_list[i][scm_list[i].obs["doublet_score"] < cutoff_threshold[i],]
#     print(meta["sample"][i] + ': ' + str(scm_list[i].shape[0]) + " cells and " + str(scm_list[i].shape[1]) +' features...')  
#     print("")

## Merge into a single AnnData object and preprocess

In [12]:
scm = scm_list[0].concatenate(
    scm_list[1:],
    index_unique=None
#     join="inner"
#     batch_key="sample",
    # batch_categories=meta["GSM.accession"]
)
scm.obs_names_make_unique()
print(scm.shape)

/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


(23394, 62703)


/home/dwm269/miniconda3/envs/scm2/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


QC plotting

In [13]:
sc.settings.set_figure_params(
    fontsize=8,
    figsize=[18,4]
)

sc.pl.violin(
    scm,
    size=0.5,
    rotation=90,
#     log=True,
    keys=['pct_counts_mito'],# '',''],
    stripplot=False,
    groupby='GSM.accession'
)

KeyError: "Could not find keys '['pct_counts_mito']' in columns of `adata.obs` or in adata.var_names."

Preprocessing...

In [ ]:
sc.pp.normalize_total(
    scm, 
    target_sum=1e4
)

sc.pp.log1p(
    scm
)

sc.pp.highly_variable_genes(
    scm,
    subset=False
)

In [ ]:
sc.pp.scale(
    scm
)
sc.tl.pca(
    scm, 
    svd_solver='arpack',
    n_comps=50
)

In [ ]:
scm.shape

In [ ]:
sc.pp.neighbors(
    scm,
    n_pcs = npcs(scm, reduction="X_pca"),
    n_neighbors = 50
)

In [ ]:
sc.tl.umap(
    scm
#     group_by ="neighbors"
)

In [ ]:
sc.set_figure_params(
    figsize=(6,6),
    dpi=200
)
sc.pl.umap(
    scm,
    legend_loc = None,
    color =["GSM.accession"] #GSM.accession
)

In [ ]:
gc.collect()

Add cell cycle inference

In [ ]:
# load in cell cycle genes lists
s_genes = list(pd.read_csv("resources/gene_lists/seurat_s_genes_2019.csv")['x'])
g2m_genes = list(pd.read_csv("resources/gene_lists/seurat_g2m_genes_2019.csv")['x'])

# score and plot
sc.tl.score_genes_cell_cycle(
    scm, 
    s_genes=s_genes, 
    g2m_genes=g2m_genes, 
    use_raw=False
)

Save merged anndata object for easy loading

In [ ]:
# Save h5ad object for subsequent python analyses
scm.write("data/pyobjs/homo_sapiens_v1.h5ad")